In [1]:
import torch
from torch import nn, optim
import os
import numpy as np
import random
from torchvision import datasets, transforms
import torchvision
from torchsummary import summary

In [2]:
# path to the location of dataset stored in my local
path = "D:/Datasets/ShapesCNN/shapes/shapes/"
os.chdir(path)

In [3]:
device = 'cpu'
train_path = os.path.join(path, 'training_set')
test_path = os.path.join(path, 'test_set')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.RandomRotation((30, 60)),
])
trainset = datasets.ImageFolder(train_path, transform = transform)
testset = datasets.ImageFolder(test_path, transform = transform)

In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4, shuffle = True)

In [6]:
images, labels = next(iter(trainloader))

In [7]:
print(images.shape, labels.shape)

torch.Size([4, 3, 28, 28]) torch.Size([4])


In [8]:
## cross entropy loss 
#loss = nn.CrossEntropyLoss()
#inps = torch.randn(3,5, requires_grad = True)
#target = torch.empty(3, dtype = torch.long).random_(5)
#output = loss(inps, target)
#print(output)

In [9]:
class Net(nn.Module):
    def __init__(self, in_channels, num_classes = 3):
        super(Net, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels, 4, kernel_size = 3),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(4, 4, kernel_size = 3),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2,2),
            nn.Flatten()
        )
        
        self.linear_layer = nn.Sequential(
            nn.Linear(100, 50),
            nn.Linear(50, 25),
            nn.Linear(25, num_classes),
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.linear_layer(x)
        return x

In [10]:
# temp_input = torch.zeros(4, 3, 28, 28)
# model = Net(in_channels = 3).to(device)
# model(temp_input)

In [11]:
model = Net(in_channels = 3).to(device)

In [12]:
learning_rate = 0.001
num_epochs = 300

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
%%time
train_loss = []
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(trainloader):
        data = data.to(device)
        target = target.to(device)
        
        scores = model(data)
        loss = criterion(scores, target)
        train_loss.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print("Epoch : {}, loss : {}".format(epoch, round(loss.item(), 3)))

Epoch : 19, loss : 0.492
Epoch : 39, loss : 1.369
Epoch : 59, loss : 0.455
Epoch : 79, loss : 0.511
Epoch : 99, loss : 1.918
Epoch : 119, loss : 1.53
Epoch : 139, loss : 0.195
Epoch : 159, loss : 0.628
Epoch : 179, loss : 0.246
Epoch : 199, loss : 0.875
Epoch : 219, loss : 0.082
Epoch : 239, loss : 0.51
Epoch : 259, loss : 1.193
Epoch : 279, loss : 0.283
Epoch : 299, loss : 0.603
Wall time: 2min 19s


In [15]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print("Accuracy : ", float(num_correct) / float(num_samples))

In [16]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Accuracy :  0.8428571428571429
Accuracy :  0.7444444444444445


In [20]:
if not os.path.exists('/models'):
    os.mkdir("models")
else:
    print("Folder exists")

In [30]:
torch.save(model.state_dict(), 'models/shape_25052021.pth')

In [34]:
# exporting to onnx model
trained_model = Net(in_channels = 3)
trained_model.load_state_dict(torch.load('models/shape_25052021.pth'))

<All keys matched successfully>

In [39]:
# 1 RGB 28 x 28 image is dummy input to model
dummy_input = torch.randn(1, 3, 28, 28)
torch.onnx.export(trained_model, dummy_input, "models/shape_25052021.onnx")